In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#from stem import Signal
#from stem.control import Controller

import pandas as pd
import time
from datetime import datetime
import re

In [2]:
# Selenium settings
options_ = Options()
options_.add_argument("--headless=new") # Chrome in headless mode (doesn't open browser)
# mimic a standard browser by setting a custom user-agent:
options_.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/115.0.0.0 Safari/537.36")
#options_.add_argument("--proxy-server=socks5://127.0.0.1:9150")

# Beautiful Soup 4 settings
headers_ = { # Bypass basic antiscraping protocols
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
}

proxies_ = {
    'http': 'socks5://127.0.0.1:9150',
    'https': 'socks5://127.0.0.1:9150'
}

In [3]:
matches_df = pd.read_csv('D:/AI Projects/EPL Data/match_data/EPL_matches_2022-2023.csv')

In [4]:
matches_df['Home'] = matches_df['Home'].replace({ 'West Brom': 'West Bromwich Albion',
                                                   "Nott'ham Forest": 'Nottingham Forest',
                                                    'West Ham': 'West Ham United',
                                                    'Tottenham': 'Tottenham Hotspur',
                                                    'Newcastle Utd': 'Newcastle United',
                                                    'Brighton': 'Brighton & Hove Albion',
                                                    'Manchester Utd': 'Manchester United',
                                                    'Sheffield Utd': 'Sheffield United',
                                                    'Wolves': 'Wolverhampton Wanderers'})

matches_df['Away'] = matches_df['Away'].replace({ 'West Brom': 'West Bromwich Albion',
                                                   "Nott'ham Forest": 'Nottingham Forest',
                                                    'West Ham': 'West Ham United',
                                                    'Tottenham': 'Tottenham Hotspur',
                                                    'Newcastle Utd': 'Newcastle United',
                                                    'Brighton': 'Brighton & Hove Albion',
                                                    'Manchester Utd': 'Manchester United',
                                                    'Sheffield Utd': 'Sheffield United',
                                                    'Wolves': 'Wolverhampton Wanderers'})

In [5]:
matches_df.head()

,Date,Season,Round,Time,Home,Away,Result
0,2022-08-05,2022-2023,1,20:00,Crystal Palace,Arsenal,A
1,2022-08-06,2022-2023,1,12:30,Fulham,Liverpool,D
2,2022-08-06,2022-2023,1,15:00,Tottenham Hotspur,Southampton,H
3,2022-08-06,2022-2023,1,15:00,Newcastle United,Nottingham Forest,H
4,2022-08-06,2022-2023,1,15:00,Leeds United,Wolverhampton Wanderers,H


In [6]:
matches_df.tail()

,Date,Season,Round,Time,Home,Away,Result
375,2023-05-28,2022-2023,38,16:30,Everton,Bournemouth,H
376,2023-05-28,2022-2023,38,16:30,Leicester City,West Ham United,H
377,2023-05-28,2022-2023,38,16:30,Aston Villa,Brighton & Hove Albion,H
378,2023-05-28,2022-2023,38,16:30,Leeds United,Tottenham Hotspur,A
379,2023-05-28,2022-2023,38,16:30,Brentford,Manchester City,H


In [7]:
matches_df.shape

(380, 7)

In [8]:
matches_df['Home'].unique()

array(['Crystal Palace', 'Fulham', 'Tottenham Hotspur',
       'Newcastle United', 'Leeds United', 'Bournemouth', 'Everton',
       'Leicester City', 'Manchester United', 'West Ham United',
       'Aston Villa', 'Manchester City', 'Southampton',
       'Wolverhampton Wanderers', 'Arsenal', 'Brighton & Hove Albion',
       'Brentford', 'Nottingham Forest', 'Chelsea', 'Liverpool'],
      dtype=object)

In [9]:
team_station_dict = {'Liverpool': 'EGGP', 
             'West Ham United': 'EGLC',
             'Fulham': 'EGLC',
             'Nottingham Forest': 'EGNX',
             'Luton Town':'EGGW',
             'Ipswich Town':'EGMC',
             'Brentford': 'EGLC', 
             'Leeds United':'EGNM',
             'West Bromwich Albion':'EGBB',
             'Burnley': 'EGNM', 
             'Watford': 'EGLL',
            'Crystal Palace': 'EGLC', 
            'Bournemouth': 'EGHH', 
            'Tottenham Hotspur': 'EGLC',
            'Newcastle United': 'EGNT', 
            'Leicester City': 'EGNX', 
            'Manchester United': 'EGCC',
            'Arsenal': 'EGLC', 
            'Norwich City': 'EGSH', 
            'Aston Villa': 'EGBB', 
            'Brighton & Hove Albion': 'EGKK',
            'Southampton': 'EGHI', 
            'Everton': 'EGGP', 
            'Manchester City': 'EGCC', 
            'Sheffield United': 'EGCN',
            'Chelsea': 'EGLC', 
            'Wolverhampton Wanderers': 'EGBB'}

# Weather Data URL:
url0 = f'https://www.wunderground.com/history/daily/EGHI/date/2019-4-5'

In [10]:
url_dict = {'Date':[],'Location':[], 'Link':[]}

for i in matches_df.index:
    location = matches_df['Home'][i]
    url_dict['Location'].append(location)
    
    station = team_station_dict[location]
    
    date = matches_df['Date'][i]
    url_dict['Date'].append(date)

    url1 = f'https://www.wunderground.com/history/daily/{station}/date/{date}'
    url_dict['Link'].append(url1)

link_df = pd.DataFrame(url_dict)
    

In [11]:
link_df.head()

,Date,Location,Link
0,2022-08-05,Crystal Palace,https://www.wunderground.com/history/daily/EGL...
1,2022-08-06,Fulham,https://www.wunderground.com/history/daily/EGL...
2,2022-08-06,Tottenham Hotspur,https://www.wunderground.com/history/daily/EGL...
3,2022-08-06,Newcastle United,https://www.wunderground.com/history/daily/EGN...
4,2022-08-06,Leeds United,https://www.wunderground.com/history/daily/EGN...


In [12]:
link_df.shape

(380, 3)

In [20]:
link_df['Link'][157]

'https://www.wunderground.com/history/daily/EGGP/date/2022-12-30'

In [ ]:
# New Dataframe to store weather data
weather_df = pd.DataFrame()

# parse links:
for l in link_df.index:
    print(f'Scraping Link: {l}')

    url2 = link_df['Link'][l]
    locat_ = link_df['Location'][l]
    date_ = link_df['Date'][l]

    # Switch to Selenium for scraping
    driver = webdriver.Chrome(options=options_) # Ensure the ChromeDriver installed 

    driver.get(url2)
    time.sleep(3) # Wait for page to load

    page_source = driver.page_source # get page source
    #driver.quit()

    soup = BeautifulSoup(page_source, 'html.parser')

    tables = soup.find_all('table') #, class_='mat-table cdk-table mat-sort ng-star-inserted')


    if tables:

        if len(tables) == 2:
            weather_table = tables[1].find('tbody', class_='mdc-data-table__content') 
        else:
            weather_table = tables[0].find('tbody', class_='mdc-data-table__content') 

        # Weather Dictionary:
        weather_dict = {'Date':[],'Location':[],'Time':[],'Temperature(°F)':[],'Dew_Point(°F)':[],
        'Humidity(%)':[],'Wind_Dir':[],'Wind_Speed(mph)':[],'Wind_Gust(mph)':[],
        'Pressure(in)':[],'Precip.(in)':[],'Condition':[]}

        rows = weather_table.find_all('tr')

        for r in rows:
            cells = r.find_all('td')

            weather_dict['Date'].append(date_)
            weather_dict['Location'].append(locat_)

            time_ = cells[0].text
            _24hr_time = datetime.strptime(time_, "%I:%M %p").strftime("%H:%M") #convert to 24hr clock
            weather_dict['Time'].append(_24hr_time)

            temperature_ = cells[1].text.split()[0]
            weather_dict['Temperature(°F)'].append(temperature_)

            dew = cells[2].text.split()[0]
            weather_dict['Dew_Point(°F)'].append(dew)

            humi_ = cells[3].text.split()[0]
            weather_dict['Humidity(%)'].append(humi_)

            wind_dir = cells[4].text
            weather_dict['Wind_Dir'].append(wind_dir)

            wind_speed = cells[5].text.split()[0]
            weather_dict['Wind_Speed(mph)'].append(wind_speed)

            wind_gust = cells[6].text.split()[0]
            weather_dict['Wind_Gust(mph)'].append(wind_gust)

            press_ = cells[7].text.split()[0]
            weather_dict['Pressure(in)'].append(press_)

            precip_ = cells[8].text.split()[0]
            weather_dict['Precip.(in)'].append(precip_)

            cond_ = cells[9].text
            weather_dict['Condition'].append(cond_)
        
    else:
        continue

    driver.quit()

    Dict_df = pd.DataFrame(weather_dict)
    weather_df = pd.concat([weather_df, Dict_df], ignore_index=True)


In [22]:
weather_df.shape

(48, 12)

In [23]:
weather_df.head()

,Date,Location,Time,Temperature(°F),Dew_Point(°F),Humidity(%),Wind_Dir,Wind_Speed(mph),Wind_Gust(mph),Pressure(in),Precip.(in),Condition
0,2022-12-30,Liverpool,01:20,41,37,87,SSE,10,0,29.38,0.0,Fair
1,2022-12-30,Liverpool,01:50,0,0,0,S,12,0,29.35,0.0,Mostly Cloudy
2,2022-12-30,Liverpool,02:20,0,0,0,SSE,14,0,29.32,0.0,Mostly Cloudy
3,2022-12-30,Liverpool,02:50,0,0,0,SE,13,0,29.29,0.0,Cloudy
4,2022-12-30,Liverpool,03:20,0,0,0,SSE,14,28,29.29,0.0,Cloudy


In [ ]:
season = matches_df['Season'][0]
weather_df.to_csv(f'D:/AI Projects/EPL Data/weather/Season_{season}_weather.csv', index=False)

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Doncaster%2C%20England%2C%20United%20Kingdom/2023-08-12/2024-05-19?unitGroup=us&elements=datetime%2Ctemp%2Cdew%2Chumidity%2Cprecip%2Cwindgust%2Cwindspeed%2Cwinddir%2Cpressure%2Cconditions&key=PPYVEQLJYFC36H234ZDRW6E6R&contentType=csv

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Doncaster%2C%20England%2C%20United%20Kingdom/2023-08-12/2024-05-19?unitGroup=us&elements=datetime%2Ctemp%2Cdew%2Chumidity%2Cprecip%2Cwindgust%2Cwindspeed%2Cwinddir%2Cpressure%2Cconditions&include=minutes&key=PPYVEQLJYFC36H234ZDRW6E6R&contentType=csv